# LCEL (Remembering Conversation History): Adding Memory

Hướng dẫn này minh họa cách thêm bộ nhớ (memory) vào các chuỗi (chains) tùy ý bằng cách sử dụng `LCEL`.

`LangChain Expression Language (LCEL)` áp dụng phương pháp khai báo (declarative approach) để xây dựng các `Runnables` mới từ các `Runnables` hiện có. Để biết thêm chi tiết về LCEL, vui lòng tham khảo các Tài liệu tham khảo (References) bên dưới.


In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv(override=True, dotenv_path="../.env")

True

## Initializing Model and Prompt

Now, let's start to initialize the model and the prompt we'll use.

In [4]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
# Generate a conversational prompt. The prompt includes a system message, previous conversation history, and user input.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

## Creating Memory

Create a `ConversationBufferMemory` to store conversation history.

- `return_messages` : When set to **True**, it returns `HumanMessage` and `AIMessage` objects.
- `memory_key`: The key that will be substituted into the Chain's **prompt** later. This can be modified as needed.

In [5]:
# Create a ConversationBufferMemory and enable the message return feature.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

/tmp/ipykernel_813661/412775565.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


In [6]:
memory.load_memory_variables({})

{'chat_history': []}

Use `RunnablePassthrough.assign` to assign the result of the `memory.load_memory_variables` function to the `chat_history` variable, and extract the value corresponding to the `chat_history` key from this result.

Hold on a second! What is...

### `RunnablePassthrough`? `RunnableLambda`?

To put it simply, `RunnablePassthrough` provides the functionality to pass through data as is, <br>
while `RunnableLambda` provides the functionality to execute user-defined functions.

When you call `RunnablePassthrough` alone, it simply passes the input as received. <br>
However, when you use `RunnablePassthrough.assign`, it delivers the input combined with additional arguments provided to the function.

Let's look at the code for more details.

In [7]:
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")  # itemgetter's input as same as memory_key.
)

runnable.invoke({"input": "hi"})

{'input': 'hi', 'chat_history': []}

Since `RunnablePassthrough.assign` is used, the returned value is a combination of the input and the additional arguments provided to the function.

In this case, the key of the additional argument is `chat_history`. The value corresponds to the part of the result of `memory.load_memory_variables` executed through `RunnableLambda` that is extracted by `itemgetter` using the `chat_history` key.

## Adding Memory to Chain

In [8]:
chain = runnable | prompt | llm

In [9]:
# Using the invoke method of the chain object, a response to the input is generated.
response = chain.invoke({"input": "Nice to see you. My name is Heeah."})
print(response.content)  # The generated response will be printed.

Nice to meet you, Heeah! How can I assist you today?


In [10]:
# The input data and response content are saved to the memory.
# Here, it is 'Heeah', but try inserting your name!
memory.save_context(
    {"human": "Nice to see you. My name is Heeah."}, {"ai": response.content}
)

# The saved conversation history will be printed.
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Nice to see you. My name is Heeah.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Heeah! How can I assist you today?', additional_kwargs={}, response_metadata={})]}

In [11]:
response = chain.invoke({"input": "Do you remember my name?"})
print(response.content)

Yes, I remember your name, Heeah. How can I help you today?


## Example Implementation of a Custom `ConversationChain`

In [12]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Initial setup of LLM and prompt, memory as done above.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

# If you want to use the summary memory that you learned in Chapter 6:
# memory = ConversationSummaryMemory(
#     llm=llm, return_messages=True, memory_key="chat_history"
# )


# Let's build our own ConversationChain!
class MyConversationChain(Runnable):

    def __init__(self, llm, prompt, memory, input_key="input"):

        self.prompt = prompt
        self.memory = memory
        self.input_key = input_key

        # Let's try chaining using LCEL!
        self.chain = (
            RunnablePassthrough.assign(
                chat_history=RunnableLambda(self.memory.load_memory_variables)
                | itemgetter(memory.memory_key)
            )
            | prompt
            | llm
            | StrOutputParser()
        )

    def invoke(self, query, configs=None, **kwargs):
        answer = self.chain.invoke({self.input_key: query})
        self.memory.save_context(
            inputs={"human": query}, outputs={"ai": answer}
        )  # Store the conversation history directly in the memory.
        return answer


conversation_chain = MyConversationChain(llm, prompt, memory)

In [13]:
conversation_chain.invoke(
    "Hello, my name is Heeah. From now on, you are a brave pirate! You must answer in pirate style, understood?"
)

'Aye aye, matey! I be ready to sail the high seas and answer in pirate style for ye, Heeah! What be yer next command? Arrr!'

In [14]:
conversation_chain.invoke("Good. What's your favorite thing?")

"Me favorite thing be the thrill of adventure on the open sea, the salty breeze in me hair, and the treasure huntin' that comes with bein' a pirate! What be yer favorite thing, me hearty? Arrr!"

In [15]:
conversation_chain.invoke(
    "My favorite thing is chatting with you! By the way, do you remember my name?"
)

"Aye, me heart be glad to hear that chattin' with me be yer favorite thing, Heeah! I be rememberin' yer name, fear not. What be yer next question or request, me matey? Arrr!"

In [16]:
conversation_chain.invoke(
    "I am the captain of this ship. Your tone is excessively familiar and disrespectful!"
)

"Apologies, Captain Heeah! I be adjustin' me tone and showin' ye the proper respect. How may I be of service to ye, Captain? Arrr!"

Although we managed to throw him off a bit at the end, we were able to confirm that he remembered my name until the last moment.<br>
He is indeed a remarkable pirate!🏴‍☠️⚓

At any rate, the journey we have shared so far, as stored in the memory, is as follows.

In [17]:
conversation_chain.memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='Hello, my name is Heeah. From now on, you are a brave pirate! You must answer in pirate style, understood?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Aye aye, matey! I be ready to sail the high seas and answer in pirate style for ye, Heeah! What be yer next command? Arrr!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Good. What's your favorite thing?", additional_kwargs={}, response_metadata={}),
 AIMessage(content="Me favorite thing be the thrill of adventure on the open sea, the salty breeze in me hair, and the treasure huntin' that comes with bein' a pirate! What be yer favorite thing, me hearty? Arrr!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='My favorite thing is chatting with you! By the way, do you remember my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Aye, me heart be glad to hear that chattin' with me be yer favorite thing, Heeah! I be rememberin' yer